In [1]:
# Welcome to the Kalshi REST v2 Starter Code!

# pypi client: recommended for more advanced programmers
#import kalshi_python

# starter client: recommended for all levels of programming experience (what this client is implemented using)
from KalshiClientsBaseV2ApiKey import ExchangeClient
import time
import json
import uuid

In [2]:
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend

def load_private_key_from_file(file_path):
    with open(file_path, "rb") as key_file:
        private_key = serialization.load_pem_private_key(
            key_file.read(),
            password=None,  # or provide a password if your key is encrypted
            backend=default_backend()
        )
    return private_key

In [3]:
# To start off, you need to have created an account at https://kalshi.com (Production) 
# or an account on the Demo https://demo.kalshi.co/

prod_key_id = "" # change these to be your personal credentials
prod_private_key = ""

demo_key_id = "" # change these to be your personal credentials
demo_private_key = load_private_key_from_file('kalshi-key.key')

# for prod
prod_api_base = "https://trading-api.kalshi.com/trade-api/v2"

# for demo
demo_api_base = "https://demo-api.kalshi.co/trade-api/v2"

## if wanting to test in prod:
# exchange_client = ExchangeClient(exchange_api_base=prod_api_base, key_id = prod_key_id, private_key = prod_private_key)

## if wanting to test in demo
exchange_client = ExchangeClient(exchange_api_base = demo_api_base, key_id = demo_key_id, private_key = demo_private_key)

# first we will check on the exchange status to confirm you are properly connected...
print(exchange_client.get_exchange_status())

msg_string 1724881863381GET/trade-api/v2/exchange/status
signature gLSMHXzi4EdEMK5RHIfxgdiJyBPIv6MsH3K7LlVraoEXGE148FGiFMoW0IBVc7baF7nWTNnza4sxSckMgtQMG2UPbymGQ7Byqa1agHlolvrExGg7U9j1POwNIrnV5Tp8MSErq0kbDAeznfGCnusabRtlM2wyW4JHqOzdx931UM0FyjxPzCjh94nRWgO0Ysfgo7noqi7k9wE8lsWGIJeMT6kvfduyyGiegtoMCUldeLLFAPC2YM/TK3inKAlEFuyzeXNezhzPa7qMt51EAIYZ8N41Tb3tTEAIotgvjkpTAFdjZDZMDWBZJw7dR+1vdLHWPJx0j0oVS3wiJuHZiy9vjw==
{'exchange_active': True, 'trading_active': True}


In [4]:
# You can discover markets through the get_markets endpoint...

# and use query parameters to filter your search!
market_params = {'limit':100,
                    'cursor':None, # passing in the cursor from the previous get_markets call
                    'event_ticker': None,
                    'series_ticker':None,
                    'max_close_ts':None, # pass in unix_ts
                    'min_close_ts':None, # pass in unix_ts
                    'status':None,
                    'tickers':None}

markets_response = exchange_client.get_markets(**market_params)
cursor = markets_response['cursor']

print('keys:', markets_response.keys())
print()
print('number of objects:', len(markets_response['markets'])) # 100 objects!
print()
print('first market in payload:', markets_response['markets'][0])
print()
print('cursor:', cursor)

msg_string 1724881871504GET/trade-api/v2/markets
signature Zl7B5SMhYOVLHCDLh004wxQsyn94G0AeTAshBLypew6wlxvQt6zQW0oqkm+EEQFS7HIlon1LQKGm+VvUTTsVo+5YCv53XDDFVwstGU2wl+pCv5mI2NxPSovcSCi4/0ARMK05V4DQ+dq+Lwf7iC30xlPuF+OWAZHfDII0YstVTxEqpLf3lbcx9gw6W7M9953H1K9rtA2bTFkgusfQyxZ/DDr9DGpH6RYhDbEOx4YxBt/lCO5H4b4GHgQDM5xg+PASYxVDyYCUglXI2i+Sh9tFkQ5k1wIUeWAITSut+mzWNJmyY8tkrRqlv8b3MNpkBlGKsUv7aITJi3cbTCob/aBV7w==
keys: dict_keys(['markets', 'cursor'])

number of objects: 100

first market in payload: {'ticker': 'NASDAQ100U-24AUG29-T19350.78', 'event_ticker': 'NASDAQ100U-24AUG29', 'market_type': 'binary', 'title': 'Will the Nasdaq-100 be above 19350.78 at the end of Aug 29, 2024?', 'subtitle': '19,350.79 or above', 'yes_sub_title': '19,350.79 or above', 'no_sub_title': '19,350.79 or above', 'open_time': '2024-08-28T20:45:00Z', 'close_time': '2024-08-29T20:00:00Z', 'expected_expiration_time': '2024-08-29T23:00:00Z', 'expiration_time': '2024-09-05T23:00:00Z', 'latest_expiration_time': '2024-09-05T23:0

In [5]:
# What are cursors and how do they work?
    
# The Cursor represents a pointer to the next page of records in the pagination.
# So this optional parameter, when filled, should be filled with the cursor string returned in a previous request to this end-point.
# Filling this would basically tell the api to get the next page containing the number of records passed on the limit parameter.
# On the other side not filling it tells the api you want to get the first page for another query.
# The cursor does not store any filters, so if any filter parameters like tickers, max_ts or min_ts were passed in the original query they must be passed again.

# Let's try it in action! Suppose we wanted to get the next 100 market objects...

market_params = {'limit':100,
                    'cursor':cursor, # passing in the cursor from the previous get_markets call
                    'event_ticker': None,
                    'series_ticker': None,
                    'max_close_ts': None, # pass in unix_ts
                    'min_close_ts': None, # pass in unix_ts
                    'status': None,
                    'tickers':None}

markets_response = exchange_client.get_markets(**market_params)
cursor = markets_response['cursor']

print('keys:', markets_response.keys())
print()
print('number of objects:', len(markets_response['markets'])) # 100 objects!
print()
print('first market in market_response payload:', markets_response['markets'][0]) # new markets!
print()
print('new cursor!', cursor)

msg_string 1724881874725GET/trade-api/v2/markets
signature OLTtRHSz9Bq/mzH/5yvhBbM5R8G6fyrLsYuo5Ds1nJnZoM6rOzHN0rdF7jEeMMGGifpWrlLvlfYjzYHmj/wcLtjuKbh+mCqV9gjUjvnOv7+LeHD2eKoMH49NoaPFbgCvUYKO+oaL5ssRxgC+sjSqm6xMfIHXs+n7oaEAMdrYlNRd+lihAtziLzghXmfJi3M2B8fJW4KEiE3+JFZ1MwGd7lYLTu/LKOjWixZyo0c86UZzIgqU8u7Kd2YhYKWZs+tpM3RoOYS/tbYSzg6r7wVijpuX4DMXrsiwuVs8dQkmXtz1W39AZr4IpjtFYwhhWuwWTzn1ANglAYgCtzpgmq/eeA==
keys: dict_keys(['markets', 'cursor'])

number of objects: 100

first market in market_response payload: {'ticker': 'AUDUSDH-24AUG2817-T0.64199', 'event_ticker': 'AUDUSDH-24AUG2817', 'market_type': 'binary', 'title': 'AUD/USD price on Aug 28, 2024 at 5pm EDT?', 'subtitle': '0.64200 or above', 'yes_sub_title': '0.64200 or above', 'no_sub_title': '0.64200 or above', 'open_time': '2024-08-28T13:03:02Z', 'close_time': '2024-08-28T21:00:00Z', 'expected_expiration_time': '2024-08-28T21:00:00Z', 'expiration_time': '2024-09-04T21:00:00Z', 'latest_expiration_time': '2024-09-04T21:00:00Z', 'settleme

In [8]:
# Next, let's look at event level data by passing an event ticker to the get_event endpoint...

event_ticker = markets_response['markets'][5]['event_ticker']
event_params = {'event_ticker': event_ticker}
event_response = exchange_client.get_event(**event_params)

print('keys:', event_response.keys())
print()
print('event object:', event_response['event'])
print()
print('first market in event_response payload:', event_response['markets'][0])

keys: dict_keys(['event', 'markets'])

event object: {'event_ticker': 'GBPUSDH-24AUG2817', 'series_ticker': 'GBPUSDH', 'sub_title': 'Aug 28, 2024 at 5pm EDT', 'title': 'GBP/USD price today at 5pm EDT?', 'mutually_exclusive': False, 'category': 'Financials', 'strike_date': '2024-08-28T21:00:00Z'}

first market in event_response payload: {'ticker': 'GBPUSDH-24AUG2817-T1.29899', 'event_ticker': 'GBPUSDH-24AUG2817', 'market_type': 'binary', 'title': '', 'subtitle': '1.29900 or above', 'yes_sub_title': '1.29900 or above', 'no_sub_title': '1.29900 or above', 'open_time': '2024-08-28T13:03:01Z', 'close_time': '2024-08-28T21:00:00Z', 'expected_expiration_time': '2024-08-28T21:00:00Z', 'expiration_time': '2024-09-04T21:00:00Z', 'latest_expiration_time': '2024-09-04T21:00:00Z', 'settlement_timer_seconds': 3600, 'status': 'active', 'response_price_units': 'usd_cent', 'notional_value': 100, 'tick_size': 1, 'yes_bid': 0, 'yes_ask': 0, 'no_bid': 100, 'no_ask': 100, 'last_price': 0, 'previous_yes_bid

In [9]:
# Next, let's look at series level data by passing a series ticker to the get_series endpoint! 
series_ticker = event_response['event']['series_ticker']
series_params = {'series_ticker': series_ticker}
series_response = exchange_client.get_series(**series_params)

print('keys:', series_response.keys())
print()
print('series object:', series_response['series'])
print()

keys: dict_keys(['series'])

series object: {'ticker': 'GBPUSDH', 'frequency': 'hourly', 'title': 'GBP/USD over/under', 'category': 'Financials', 'tags': [], 'settlement_sources': [{'url': 'https://www.tradingview.com/chart/?symbol=FX_IDC%3AGBPUSD', 'name': 'ICE'}], 'contract_url': 'https://kalshi-public-docs.s3.us-east-1.amazonaws.com/regulatory/product-certifications/GBPUSD.pdf'}



In [7]:
# Next let's look at the recent market history for a market
ticker = 'NGDP-22-C7.5'

market_history_params = {'ticker': ticker,
                            'limit': 100,
                            'cursor': None,
                            'max_ts': None, # pass in unix_ts
                            'min_ts': round(time.time()-1000000) # passing a recent unix_ts
                                }
market_history_response = exchange_client.get_market_history(**market_history_params)

print('keys:', market_history_response.keys())
print()
print('most recent market history object:', market_history_response['history'][-1])
print()

# and then also look at the most current view of the orderbook
market_history_params = {'ticker':ticker,
                            'depth': 30
                                }
orderbook_response = exchange_client.get_orderbook(**market_history_params)

print('keys:', orderbook_response.keys())
print()
print('orderbook object:', orderbook_response)
print()

keys: dict_keys(['ticker', 'history', 'cursor'])

most recent market history object: {'yes_price': 23, 'yes_bid': 23, 'yes_ask': 87, 'no_bid': 13, 'no_ask': 77, 'volume': 2761, 'open_interest': 1644, 'ts': 1669651310}

keys: dict_keys(['orderbook'])

orderbook object: {'orderbook': {'yes': [[1, 515], [23, 2503]], 'no': [[1, 1146], [2, 152], [3, 38], [6, 300], [7, 38], [12, 62], [13, 154]]}}



In [10]:
# Now let's suppose we wanted to place a trade on one of these markets... 
# to do so, we would first want to check out available balance...

current_balance = exchange_client.get_balance()
current_balance

{'balance': 44856}

In [11]:
# Now that you have some balance, you might want to see how your current positions are doing...

positions_params = {'limit': None,
                        'cursor': None,
                        'settlement_status': None,
                        'ticker': None,
                        'event_ticker': None}

current_position = exchange_client.get_positions(**positions_params)
current_position

{'cursor': '',
 'market_positions': [{'ticker': 'BTCMAXY-24DEC31-90000',
   'total_traded': 5100,
   'position': 100,
   'market_exposure': 5100,
   'realized_pnl': 0,
   'resting_orders_count': 0,
   'fees_paid': 175,
   'last_updated_ts': '2024-06-03T18:04:41.052476Z'},
  {'ticker': 'BTCMAXY-24DEC31-80000',
   'total_traded': 9450,
   'position': 150,
   'market_exposure': 9450,
   'realized_pnl': 0,
   'resting_orders_count': 0,
   'fees_paid': 80,
   'last_updated_ts': '2024-07-31T20:13:10.833106Z'},
  {'ticker': 'LLM1-24DEC31-OAI',
   'total_traded': 1206,
   'position': 21,
   'market_exposure': 1206,
   'realized_pnl': 0,
   'resting_orders_count': 0,
   'fees_paid': 17,
   'last_updated_ts': '2024-06-20T22:39:35.771596Z'},
  {'ticker': 'LLM1-24DEC31-A',
   'total_traded': 1950,
   'position': 50,
   'market_exposure': 1950,
   'realized_pnl': 0,
   'resting_orders_count': 0,
   'fees_paid': 84,
   'last_updated_ts': '2024-07-29T15:07:29.283082Z'},
  {'ticker': 'CPICORE-24JUN-T0

In [12]:
# seems like some of your recent orders had been filled. To check on those we use the get_positions endpoint ...

fills_params = {'ticker':None,
                    'order_id':None,
                    'min_ts':None,
                    'max_ts':None,
                    'limit': None,
                    'cursor': None}

fills = exchange_client.get_fills(**fills_params)
fills

{'fills': [{'trade_id': '2a68e347-325f-4801-af09-516bbcd17fb3',
   'ticker': 'BTCMAXY-24DEC31-80000',
   'order_id': '149b060c-e263-4e6f-9a7b-97fd468f3900',
   'side': 'yes',
   'action': 'buy',
   'count': 100,
   'yes_price': 62,
   'no_price': 38,
   'is_taker': False,
   'created_time': '2024-07-31T20:13:10.833106Z'},
  {'trade_id': '0801aca3-6230-4093-a699-a6b22908bd97',
   'ticker': 'BTCMAXY-24DEC31-80000',
   'order_id': 'f5d8a8a6-b9ab-46b9-a6d6-88a1b7b16798',
   'side': 'yes',
   'action': 'buy',
   'count': 50,
   'yes_price': 65,
   'no_price': 35,
   'is_taker': True,
   'created_time': '2024-07-31T17:34:34.116259Z'},
  {'trade_id': '63f3afbd-b82d-4526-8ae3-4d08c6933329',
   'ticker': 'LLM1-24DEC31-A',
   'order_id': '47dc838c-6501-4358-856c-dddd7b1f3d82',
   'side': 'yes',
   'action': 'buy',
   'count': 50,
   'yes_price': 39,
   'no_price': 61,
   'is_taker': True,
   'created_time': '2024-07-29T15:07:29.283082Z'},
  {'trade_id': 'f95a179c-7a91-4645-bc47-6ec3f40a848b',
  

In [11]:
# you may even want to check on some of your recent positions settled...

settlement_params = {'limit': None,
                        'cursor': None}

settlements = exchange_client.get_portfolio_settlements(**settlement_params)
settlements

{'settlements': []}

In [6]:
# Now onto placing an order...
# There are many different ways to think about placing orders at Kalshi. 
# The following param examples will walk through some of those

# Limit buy order for 10 units at 30c No on GDPW-22-A3

ticker = 'TESTING-5'

order_params = {'ticker':ticker,
                    'client_order_id':str(uuid.uuid4()),
                    'type':'limit',
                    'action':'buy',
                    'side':'no',
                    'count':10,
                    'yes_price':None, # yes_price = 100 - no_price
                    'no_price':30, # no_price = 100 - yes_price
                    'expiration_ts':None,
                    'sell_position_floor':None,
                    'buy_max_cost':None}

exchange_client.create_order(**order_params)

# EQUIVALENTLY, because buying No is equivalent to selling yes...

# order_params = {'ticker':ticker,
#                     'client_order_id':str(uuid.uuid4()),
#                     'type':'limit',
#                     'action':'sell',
#                     'side':'yes',
#                     'count':10,
#                     'yes_price':None, # yes_price = 100 - no_price
#                     'no_price':30, # no_price = 100 - yes_price
#                     'expiration_ts':None,
#                     'sell_position_floor':None,
#                     'buy_max_cost':None}

# exchange_client.create_order(**order_params)


# # Market sell order for 12 units Yes on GDPW-22-A3, without flipping position

# order_params = {'ticker':ticker,
#                     'client_order_id':str(uuid.uuid4()),
#                     'type':'market',
#                     'action':'sell',
#                     'side':'yes',
#                     'count':12,
#                     'yes_price':1,
#                     'no_price':None,
#                     'expiration_ts':None,
#                     'sell_position_floor':0,
#                     'buy_max_cost':None}

# exchange_client.create_order(**order_params)

# # EQUIVALENTLY, because buying No is equivalent to selling yes...

# order_params = {'ticker':ticker,
#                     'client_order_id':str(uuid.uuid4()),
#                     'type':'market',
#                     'action':'buy',
#                     'side':'no',
#                     'count':12,
#                     'yes_price':1,
#                     'no_price':None,
#                     'expiration_ts':None,
#                     'sell_position_floor':0,
#                     'buy_max_cost':None}

# exchange_client.create_order(**order_params)             


{'ticker': 'TESTING-5', 'client_order_id': 'bfbd2cb1-2a37-454a-9276-4700ba64f084', 'side': 'no', 'action': 'buy', 'count': 10, 'type': 'limit', 'no_price': 30}
msg_string 1724881878733POST/trade-api/v2/portfolio/orders
signature nSL5OZMoq4ijxiEOrnh4LgROWhXh4rnY+DYGPykEX8AIPl7Sz2BIjmIH8pSYUGmXwScYYMMGkyyOPXIVdSxKTaJKNS9Q7LFKxJXTUW7ij/ApRB/SmxNca7Uew7d5dfCPmr/fg5bds0y/3PuahpAGyBTCEXb7vFqFt2DxaMZP2RMs8wv6sfBJzbo4YLG0iJsxCbxpJO1Xl/Vw99kXjHs4aLrzwCkeAfj3ZrnJ9oq0ZSaOOIW3DX1CymV5BgsOxgqWxLFfTUL2A2ebBKcby1dSlTC41Nzdji3qB10At9S9SpZ53tIS5y1sQSIKuLXhNKmp3eX6Wh5hZfF84YgBJjG/5A==


{'order': {'order_id': 'a83b4604-6ad5-49e0-bf65-d89b175df537',
  'user_id': 'd6a190b7-31de-4855-9fed-e1fd364dc2cf',
  'ticker': 'TESTING-5',
  'status': 'resting',
  'yes_price': 70,
  'no_price': 30,
  'created_time': '2024-08-28T21:51:18.993589Z',
  'expiration_time': None,
  'self_trade_prevention_type': '',
  'action': 'buy',
  'side': 'no',
  'type': 'limit',
  'client_order_id': 'bfbd2cb1-2a37-454a-9276-4700ba64f084',
  'order_group_id': ''}}